# Make an SED and calculate fundamental parameters
`SEDkit` is used primarily for the creation and analysis of spectral energy distributions. The code stitches together the provided photometry and spectra and flux calibrates it given a distance. Simple integration then gives the bolometric luminosity. If radius is given or one can be estimated from a given age, effective temperature is also calculated.

This notebook will show you how to make an SED, modify its input parameters, and calculate and plot results.

### Requirements
- `astropy>=3.0.2`
- `bokeh>=0.12.6`
- `astroquery>=0.3.8`

In [1]:
# Imports
from sedkit import sed, spectrum
import astropy.units as q
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Initialize an SED

Let's create a new SED for an L3.5 brown dwarf. The software assumes a field age if no `age` argument is given.

In [2]:
s = sed.SED(name='2MASSW J0036159+182110')

# Print the metadata
print('\n',s.all_names,'\n\n',s.sky_coords)

[sedkit] SED initialized
[sedkit] Setting name to 2MASSW J0036159+182110
[sedkit] 1 record for 2MASSW J0036159+182110 found in Simbad.
[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (0.60447553, 18.35285861)>
[sedkit] There was a problem determining the interstellar reddening. Setting to 0. You can manually set this with the 'reddening' attribute.
[sedkit] Setting parallax to (<Quantity 114.4167 mas>, <Quantity 0.2088 mas>) with reference '2018yCat.1345....0G'
[sedkit] Setting distance to (<Quantity 8.74 pc>, <Quantity 0.02 pc>, <Quantity 0.02 pc>) with reference '2018yCat.1345....0G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] Setting radius to (<Quantity 0.08 solRad>, <Quantity 0. solRad>, <Quantity 0. solRad>) with reference '2013ApJS..208....9P'
[sedkit] Setting spectral_type to (73.5, 0.5, 'V', None, None) with reference '2000AJ....119..369R'

 ['USNO-B1.0 1083-00010105', '[B2006] J003616.1+182110', 'TIC 42695

Given a `name` argument, a SIMBAD query is performed and the alternate names and sky coordinates are extracted.

Alternately, the `sky_coords` argument can be given and the names are similarly retrieved from SIMBAD.

In [3]:
# Clear the names
s.all_names = []

# Add the coordinates
s.sky_coords = 9.0673755*q.deg, 18.352889*q.deg

# Print the metadata
print(s.all_names,'\n\n',s.sky_coords)

[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (9.0673755, 18.352889)>
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] 1 record for <SkyCoord (ICRS): (ra, dec) in deg
    (9.0673755, 18.352889)> found in Simbad.
[sedkit] Setting parallax to (<Quantity 114.4167 mas>, <Quantity 0.2088 mas>) with reference '2018yCat.1345....0G'
[sedkit] Setting distance to (<Quantity 8.74 pc>, <Quantity 0.02 pc>, <Quantity 0.02 pc>) with reference '2018yCat.1345....0G'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
[sedkit] Setting spectral_type to (73.5, 0.5, 'V', None, None) with reference '2000AJ....119..369R'
['USNO-B1.0 1083-00010105', '[B2006] J003616.1+182110', 'TIC 426959594', '2MASS J00361617+1821104', '2MASSW J0036162+182110', '** BNA    1A', 'WISEA J003616.79+182111.6', 'LSPM J0036+1821', 'Gaia DR2 2794735086363871360', 'WDS J00363+1821A', '2MUCD 20029', '2MASSW J0036159+182110'] 

 <SkyCoor

## Adding a radius and spectral type

Both `radius` and `spectral_type` can be explicitly set upon initialization of the SED or by setting the attributes. If a `spectral_type` is given and `radius` is not, the radius will be estimated from the spectral type according to calculated radius-spectral type relations.

In [20]:
# Clear the radius
s.radius = None

# Set the spectral type
s.spectral_type = 'L3.5'

[sedkit] Setting radius to (<Quantity 0.08 solRad>, <Quantity 0. solRad>, <Quantity 0. solRad>) with reference '2013ApJS..208....9P'
[sedkit] Setting spectral_type to (73.5, 0.5, 'V', None, None) with reference 'None'


In [21]:
# Set the radius
s.radius = 0.1*q.R_sun, 0.01*q.R_sun

[sedkit] Setting radius to (<Quantity 0.1 solRad>, <Quantity 0.01 solRad>) with reference 'None'


## Adding photometry
Now let's add some photometry and take a look at how it's stored.

You can pass a CSV file of photometry with the column names 'band', 'magnitude', and 'uncertainty' like so...

In [22]:
# Add photometry from CSV
s.add_photometry_file(resource_filename('sedkit', 'data/L3_photometry.txt'))
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

[sedkit] Setting 2MASS.H photometry to 11.588 (0.030) with reference 'None'
[sedkit] Setting 2MASS.J photometry to 12.466 (0.027) with reference 'None'
[sedkit] Setting 2MASS.Ks photometry to 11.058 (0.021) with reference 'None'
[sedkit] Setting SDSS.u photometry to 24.688 (1.389) with reference 'None'
[sedkit] Setting SDSS.g photometry to 22.524 (0.137) with reference 'None'
[sedkit] Setting SDSS.r photometry to 19.690 (0.019) with reference 'None'
[sedkit] Setting SDSS.i photometry to 17.179 (0.005) with reference 'None'
[sedkit] Setting SDSS.z photometry to 15.217 (0.005) with reference 'None'
   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u         24.69             1.389
   SDSS.g         22.53             0.137
Johnson.V         21.44             0.024
   SDSS.r         19.69             0.019
   SDSS.i         17.17             0.005
Cousins.I         20.34               nan
   SDSS.z         15.22             0.005
  2MASS.J         

Or you can add individual measurements by passing the band name, magnitude, and uncertainty to the `add_photometry()` method like so...

In [23]:
# Add individual measurement
s.add_photometry('Johnson.V', 21.43, 0.024)
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

[sedkit] Setting Johnson.V photometry to 21.430 (0.024) with reference 'None'
   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u         24.69             1.389
   SDSS.g         22.53             0.137
Johnson.V         21.44             0.024
   SDSS.r         19.69             0.019
   SDSS.i         17.17             0.005
Cousins.I         20.34               nan
   SDSS.z         15.22             0.005
  2MASS.J         12.47             0.027
  2MASS.H        11.586              0.03
 2MASS.Ks        11.055             0.021
  WISE.W1         10.54             0.024
  WISE.W2         10.24             0.019
  WISE.W3          9.85             0.054
  WISE.W4          8.17               nan


Upper limits can be indicated by passing `None`, `np.nan` or `0` as the uncertainty.

In [24]:
# Add an upper limit
s.add_photometry('Cousins.I', 20.35)
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

[sedkit] Setting Cousins.I photometry to 20.350 (nan) with reference 'None'
   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u         24.69             1.389
   SDSS.g         22.53             0.137
Johnson.V         21.44             0.024
   SDSS.r         19.69             0.019
   SDSS.i         17.17             0.005
Cousins.I         20.34               nan
   SDSS.z         15.22             0.005
  2MASS.J         12.47             0.027
  2MASS.H        11.586              0.03
 2MASS.Ks        11.055             0.021
  WISE.W1         10.54             0.024
  WISE.W2         10.24             0.019
  WISE.W3          9.85             0.054
  WISE.W4          8.17               nan


If the source has a `name` or `sky_coords`, you can query the SDSS, 2MASS, Pan-STARRS, or WISE catalogs with the `find_<catalog>()` method like so...

In [25]:
# Query WISE
s.find_WISE()
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

[sedkit] 1 record found in II/328/allwise using target name '2MASSW J0036159+182110'
[sedkit] Setting WISE.W1 photometry to 10.536 (0.024) with reference 'II/328/allwise'
[sedkit] Setting WISE.W2 photometry to 10.245 (0.019) with reference 'II/328/allwise'
[sedkit] Setting WISE.W3 photometry to 9.851 (0.054) with reference 'II/328/allwise'
[sedkit] Setting WISE.W4 photometry to 8.169 (nan) with reference 'II/328/allwise'
   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u         24.69             1.389
   SDSS.g         22.53             0.137
Johnson.V         21.44             0.024
   SDSS.r         19.69             0.019
   SDSS.i         17.17             0.005
Cousins.I         20.34               nan
   SDSS.z         15.22             0.005
  2MASS.J         12.47             0.027
  2MASS.H        11.586              0.03
 2MASS.Ks        11.055             0.021
  WISE.W1         10.54             0.024
  WISE.W2         10.24      

## Adding a distance
A `distance` or `parallax` can be added by setting the attribute with a (value, uncertainty) in `astropy.units.quantitiy.Quantity` units. The other is caluclated from whichever is set first.

In [26]:
# Set the parallax
s.parallax = 114.2*q.mas, 0.8*q.mas

# ...or the distance
# s.distance = 8.87*q.pc, 0.06*q.pc

print(s.parallax, s.distance)

[sedkit] Setting parallax to (<Quantity 114.2 mas>, <Quantity 0.8 mas>) with reference 'None'
[sedkit] Setting distance to (<Quantity 8.76 pc>, <Quantity 0.06 pc>, <Quantity 0.06 pc>) with reference 'None'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'
(<Quantity 114.2 mas>, <Quantity 0.8 mas>) (<Quantity 8.76 pc>, <Quantity 0.06 pc>, <Quantity 0.06 pc>)


Once a parallax or distance is provided, the data is automatically flux calibrated.

In [27]:
print(s.photometry[['band','app_magnitude','app_magnitude_unc']])

   band   app_magnitude app_magnitude_unc
--------- ------------- -----------------
   SDSS.u         24.69             1.389
   SDSS.g         22.53             0.137
Johnson.V         21.44             0.024
   SDSS.r         19.69             0.019
   SDSS.i         17.17             0.005
Cousins.I         20.34               nan
   SDSS.z         15.22             0.005
  2MASS.J         12.47             0.027
  2MASS.H        11.586              0.03
 2MASS.Ks        11.055             0.021
  WISE.W1         10.54             0.024
  WISE.W2         10.24             0.019
  WISE.W3          9.85             0.054
  WISE.W4          8.17               nan


Once the sky coordinates and distance are set, the dust column density is calculated from the Bayestar17 3D dust map and stored to deredden the photometry.

In [28]:
s.reddening

0.0

## Adding spectra
Spectra aren't necessary to build an SED but let's add some for fun.

Spectra can be added from an ASCII or FITS file...

In [29]:
# Add a spectrum from a file, specifying the astropy units
file = resource_filename('sedkit', 'data/L3_spectrum.fits')
s.add_spectrum_file(file, wave_units=q.um, flux_units=q.W/q.m**2/q.um, name='NIR spectrum')

# Plot it
s.spectra[0]['spectrum'].plot(draw=True)

[sedkit] Looks like that 0.81396484375 um-4.125 um spectrum is already added. Skipping...


Alternatively, you can add the spectrum manually
```
wave, flux, unc = np.genfromtxt(file, unpack=True)
s.add_spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)
```

## Calculating results
Now that our SED has all the data it needs, we can call the `results` method, which constructs the SED and caluclates the fundamental parameters.

If no `radius` is given, one will be estimated from model isochrones using the calculated `Lbol` and assumed `age`.

In [30]:
# Calculate and display the results
s.results

[sedkit] Setting fbol to (<Quantity 5.09574852e-11 erg / (cm2 s)>, <Quantity 1.66845241e-13 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 4.67872106e+29 erg / s>, <Quantity 6.58974043e+27 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-3.913, 0.006) with reference 'This Work'
[sedkit] Setting Mbol to (14.537, 0.008) with reference 'This Work'
[sedkit] Could not calculate logg without Lbol and age
(<Quantity -0.05 solMass>, <Quantity 0.01 solMass>, <Quantity 0.01 solMass>): mass value is not in valid range [0.0 solMass, 226.0 solMass].
[sedkit] Setting Teff to (<Quantity 1919 K>, <Quantity 96 K>) with reference 'This Work'


param,value,unc,units
object,object,object,object
name,2MASSW J0036159+182110,--,--
ra,9.0673755,--,--
dec,18.352889,--,--
age,--,--,--
membership,--,--,--
distance,8.76,0.06,pc
parallax,114.2,0.8,mas
SpT,L3.5V,--,--
spectral_type,73.5,0.5,--


Cool. Filippazzo et al. (2015) has this object at $1869 \pm 64$ K.

## Fitting the SED
We can also fit a blackbody to the data using a Levenberg-Marquardt least-squares minimization. The calculated `Teff` will be used as the initial guess if available. Otherwise, this can be set manually with the `Teff_init` argument.

In [31]:
# Run the blackbody fit
s.fit_blackbody()

[sedkit] Blackbody fit: 1614 K


We can also fit the spectrum to estimate a spectral type.

In [32]:
# Run the spectral type fit
s.fit_spectral_type()

[sedkit] Best fit SpeX Prism Library spec: Opt:L3


## Plotting the SED
We can also plot the SED and toggle a bunch of things on and off in the plot.

In [33]:
# Plot the SED
s.make_sed()
fig = s.plot(integral=True, blackbody=False)

[sedkit] Setting fbol to (<Quantity 5.09574852e-11 erg / (cm2 s)>, <Quantity 1.66845241e-13 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 4.67872106e+29 erg / s>, <Quantity 6.58974043e+27 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-3.913, 0.006) with reference 'This Work'
[sedkit] Setting Mbol to (14.537, 0.008) with reference 'This Work'
[sedkit] Could not calculate logg without Lbol and age
(<Quantity -0.05 solMass>, <Quantity 0.01 solMass>, <Quantity 0.01 solMass>): mass value is not in valid range [0.0 solMass, 226.0 solMass].
[sedkit] Setting Teff to (<Quantity 1919 K>, <Quantity 96 K>) with reference 'This Work'


## Fine tuning
A nice feature of `SEDkit` is that the caluclations cascade when you change an attribute.

For example, the radius esitmated from Filippazzo et al. (2015) is 0.101 Solar radii. Let's say we want to see what the results are with a slightly smaller radius. We can just set the attribute (value, uncertainty) and show the results again.

In [34]:
s.radius = 0.9*q.Rjup, 0.1*q.Rjup

[sedkit] Setting radius to (<Quantity 0.9 jupiterRad>, <Quantity 0.1 jupiterRad>) with reference 'None'


In [35]:
s.results

[sedkit] Setting fbol to (<Quantity 5.09574852e-11 erg / (cm2 s)>, <Quantity 1.66845241e-13 erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 4.67872106e+29 erg / s>, <Quantity 6.58974043e+27 erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-3.913, 0.006) with reference 'This Work'
[sedkit] Setting Mbol to (14.537, 0.008) with reference 'This Work'
[sedkit] Could not calculate logg without Lbol and age
(<Quantity -0.05 solMass>, <Quantity 0.01 solMass>, <Quantity 0.01 solMass>): mass value is not in valid range [0.0 solMass, 226.0 solMass].
[sedkit] Setting Teff to (<Quantity 1995 K>, <Quantity 111 K>) with reference 'This Work'


param,value,unc,units
object,object,object,object
name,2MASSW J0036159+182110,--,--
ra,9.0673755,--,--
dec,18.352889,--,--
age,--,--,--
membership,--,--,--
distance,8.76,0.06,pc
parallax,114.2,0.8,mas
SpT,L3.5V,--,--
spectral_type,73.5,0.5,--


So the Teff went from 1919K to 1995K, which is right since if we assume a smaller radius with the same Lbol, the Teff should be higher.

## Export the SED
Now we can export the result to a zip file (if `zipped=True`) or directory like so:

In [22]:
s.export('/Users/jfilippazzo/Desktop/', zipped=True)